In [1]:
import numpy as np 
import os
import pandas as pd

import matplotlib.pyplot as plt 

In [2]:
#make list of all datasets 
dates = ["20190904", "20190828", "20190409", "20190125", "20180807", "20180727"]
files = os.listdir("C:/Users/marij/Documents/studie/CLS/Masterproject/data_processing/data_kilosort") #kilosort output folder
#print(files)
file_list = []

for i in dates: #extract wanted files out of the folder
    file_list.append([file for file in files if i in file])
    
file_list = np.array(file_list).flatten()

bins = [str(1), str(5), str(10), str(15), str(20), str(30), str(40), str(50)]
voc_suffix = ["", "_allvoc", "_baseline", "_baseline_voclen", "_first_half_voc", "_second_half_voc", "_startvoc", "_endvoc"]
voc_cols = ["whole", "all_voc", "baseline_whole", "baseline_voclen","first_half", "second_half", "first_100ms", "last_100ms"]
trials = 5
vocs = 18
minutes = 5
parts = 2

folder_path = 'C:/Users/marij/Documents/studie/CLS/Masterproject/data_processing/data_binarization/'
save_folder = "C:/Users/marij/Documents/studie/CLS/Masterproject/RESULTS/data_information/"
#file_list = file_list[[10, 11]]
print(file_list)
#bins = [str(1)]

['20190904_RippleF2_spk2_d02.dat' '20190904_Vocalization_spk2_d02.dat'
 '20190828_RippleF2_d02.dat' '20190828_Vocalization_d02.dat'
 '20190409_Ripple2_d01.dat' '20190409_Vocalization_d01.dat'
 '20190125_Ripple2_d01.dat' '20190125_Vocalization_d01.dat'
 '20180807_Ripple2_d01.dat' '20180807_Vocalization_d01.dat'
 '20180727_Ripple2_d01.dat' '20180727_Vocalization_d01.dat']


In [3]:
for dataset in file_list:
    path = folder_path + dataset + '/'
    date = dataset[0:8]
    
    neuron_file = path + date + '_neuronlabels.txt'
    neurons = pd.read_csv(neuron_file, header = None, names = ['id', 'channel'])
    neurons_id = neurons['id'].to_numpy().flatten()
    neurons_channel = neurons['channel'].to_numpy().flatten()
    n = len(neurons)
    print('n = ', n)
    
    df = pd.DataFrame(index = bins)
    df.index.name = "bin (ms)"
    if "Vocalization" in dataset:
        print("vocalization file: ", dataset)
        save_suffix = date + "_vocalization"
        for data_type in range(len(voc_suffix)):
            bin_N = []
            for binsec in bins:
                #path to datafile
                data_path = path + date + '_binsec' + str(binsec) + voc_suffix[data_type] + '.dat'
                #print(data_path)
                data_strings = np.loadtxt(data_path, dtype=str)
                #get N
                N = data_strings.shape[0]
                bin_N.append(N)
            df[voc_cols[data_type]] = bin_N
        
        for trial in range(trials):
            bin_N = []
            for binsec in bins:
                #path to datafile
                cut_path = path + "cut_binarized_data" + binsec + '/'
                data_path = cut_path + date + '_binsec' + str(binsec) + "_trial" + str(trial+1) + '.dat'
                #print(data_path)
                data_strings = np.loadtxt(data_path, dtype=str)
                #get N
                N = data_strings.shape[0]
                bin_N.append(N)
            col_title = "trial" + str(trial+1)
            df[col_title] = bin_N
        
        for voc in range(vocs):
            bin_N = []
            for binsec in bins:
                #path to datafile
                cut_path = path + "cut_binarized_data" + binsec + '/'
                data_path = cut_path + date + '_binsec' + str(binsec) + "_voc" + str(voc+1) + '.dat'
                #print(data_path)
                data_strings = np.loadtxt(data_path, dtype=str)
                #get N
                N = data_strings.shape[0]
                bin_N.append(N)
            col_title = "voc" + str(voc+1)
            df[col_title] = bin_N
            
    elif "Ripple" in dataset:
        print("Ripple file: ", dataset)
        save_suffix = date + "_ripple"
        bin_N = []
        for binsec in bins:
            #path to datafile
            data_path = path + date + '_binsec' + str(binsec) + '.dat'
            #print(data_path)
            data_strings = np.loadtxt(data_path, dtype=str)
            #get N
            N = data_strings.shape[0]
            bin_N.append(N)
        df["whole"] = bin_N
        
        for minute in range(minutes):
            bin_N = []
            for binsec in bins:
                #path to datafile
                cut_path = path + "cut_binarized_data" + binsec + '/'
                data_path = cut_path + date + '_binsec' + str(binsec) + "_minute" + str(minute+1) + '.dat'
                #print(data_path)
                data_strings = np.loadtxt(data_path, dtype=str)
                #get N
                N = data_strings.shape[0]
                bin_N.append(N)
            col_title = "minute" + str(minute+1)
            df[col_title] = bin_N
        
        for part in range(parts):
            bin_N = []
            for binsec in bins:
                #path to datafile
                cut_path = path + "cut_binarized_data" + binsec + '/'
                data_path = cut_path + date + '_binsec' + str(binsec) + "_ripple" + str(part+1) + '.dat'
                #print(data_path)
                data_strings = np.loadtxt(data_path, dtype=str)
                #get N
                N = data_strings.shape[0]
                bin_N.append(N)
            col_title = "part" + str(part+1)
            df[col_title] = bin_N
    else:
        print("weird dataset")
    print(df)
    #make save_path
    save_path = save_folder + save_suffix
    textfile = save_path + '.txt'
    #f = open(textfile, 'w')
    #f.write(dataset)
    #f.write('\n')
    #fwrite = "n = " + str(n) + "\n"
    #f.write(fwrite)
    #column_names = df.columns.values.tolist()
    #column_names = ["bin(ms)"] + column_names
    #column_names_text = ','.join(column_names)
    #f.write(column_names_text)
    #f.write("\n")
    df.to_csv(textfile, mode='w', header = True, index = True)
    #f.close()

n =  16
Ripple file:  20190904_RippleF2_spk2_d02.dat
           whole  minute1  minute2  minute3  minute4  minute5   part1   part2
bin (ms)                                                                     
1         756569   119971   119972   119985   119982   119974  301395  301399
5         151314    23996    23996    23998    23999    23996   60280   60280
10         75657    11999    11999    12000    12001    11999   30141   30140
15         50438     8000     8000     8001     8001     8000   20094   20094
20         37829     6000     6001     6001     6001     6001   15071   15070
30         25219     4001     4001     4001     4002     4001   10048   10047
40         18915     3000     3001     3001     3001     3001    7536    7535
50         15132     2401     2402     2402     2402     2402    6029    6029
n =  17
vocalization file:  20190904_Vocalization_spk2_d02.dat
           whole  all_voc  baseline_whole  baseline_voclen  first_half  \
bin (ms)                      

           whole  minute1  minute2  minute3  minute4  minute5   part1   part2
bin (ms)                                                                     
1         658758   119935   119928   119973   119981   119882  301369  301370
5         131752    23990    23988    23997    23998    23978   60275   60275
10         65876    11996    11995    11999    11999    11990   30138   30138
15         43918     7998     7997     8000     8000     7994   20093   20092
20         32938     5999     5999     6001     6001     5997   15070   15069
30         21959     4000     3999     4000     4000     3998   10047   10047
40         16469     3000     3001     3001     3001     3000    7535    7535
50         13176     2401     2401     2402     2402     2400    6029    6028
n =  13
vocalization file:  20190125_Vocalization_d01.dat
           whole  all_voc  baseline_whole  baseline_voclen  first_half  \
bin (ms)                                                                 
1         5750